# Ensemble Learning

## Initial Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.ensemble import EasyEnsembleClassifier

## Read the CSV and Perform Basic Data Cleaning

In [4]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)
# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


## Split the Data into Training and Testing

In [5]:
# Create our features

lending_data_binary_encoded = pd.get_dummies(df, columns=["home_ownership",
                                                          "verification_status",
                                                         "hardship_flag",
                                                         "debt_settlement_flag",
                                                          "pymnt_plan",
                                                          "initial_list_status",
                                                          "issue_d",
                                                          "next_pymnt_d",
                                                          "application_type"
                                                         ])
lending_data_binary_encoded.head()
X = lending_data_binary_encoded.copy()
X.drop("loan_status", axis=1, inplace=True)

# Create our target
y = lending_data_binary_encoded[["loan_status"]]

In [6]:
lending_data_binary_encoded.head()

,loan_amnt,int_rate,installment,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,pymnt_plan_n,initial_list_status_f,initial_list_status_w,issue_d_Feb-2019,issue_d_Jan-2019,issue_d_Mar-2019,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App
0,10500.0,0.1719,375.35,66000.0,low_risk,27.24,0.0,0.0,8.0,0.0,...,1,0,1,0,0,1,0,1,1,0
1,25000.0,0.2000,929.09,105000.0,low_risk,20.23,0.0,0.0,17.0,1.0,...,1,0,1,0,0,1,0,1,1,0
2,20000.0,0.2000,529.88,56000.0,low_risk,24.26,0.0,0.0,8.0,0.0,...,1,0,1,0,0,1,0,1,1,0
3,10000.0,0.1640,353.55,92000.0,low_risk,31.44,0.0,1.0,10.0,1.0,...,1,0,1,0,0,1,0,1,1,0
4,22000.0,0.1474,520.39,52000.0,low_risk,18.76,0.0,1.0,14.0,0.0,...,1,0,1,0,0,1,0,1,1,0


In [7]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,pymnt_plan_n,initial_list_status_f,initial_list_status_w,issue_d_Feb-2019,issue_d_Jan-2019,issue_d_Mar-2019,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,1.0,0.123879,0.876121,0.371696,0.451066,0.177238,0.383161,0.616839,0.860340,0.139660
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.0,0.329446,0.329446,0.483261,0.497603,0.381873,0.486161,0.486161,0.346637,0.346637
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,1.0,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,1.0,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,1.0,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [8]:
# Check the balance of our target values
y['loan_status'].value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [9]:
# Split the X and y into X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [10]:
# Create the StandardScaler instance
scaler = StandardScaler()

In [11]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [12]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [13]:
#this dictionary will hold the accuracy score for different models
balanced_accuracy_score_results ={}
#this function will be used to execute the logistic regression model on training data
def execute_classification_model(model,
                                 model_name,
                                 X_train,
                                 y_train,
                                 X_test,
                                 y_test,
                                 solver='lbfgs',
                                 random_state=1):
    result_list=[]
    model.fit(X_train, y_train)
    # Calculated the balanced accuracy score
    y_pred = model.predict(X_test)
    bal_accuracy_score= balanced_accuracy_score(y_test, y_pred)
    print(bal_accuracy_score)
    balanced_accuracy_score_results[model_name]=bal_accuracy_score
    # store the result of confusion matrix in variable
    matrix = confusion_matrix(y_test, y_pred)
    # Print the imbalanced classification report
    print(classification_report_imbalanced(y_test, y_pred))
    #store result of counter of y_train in variable
    counter = Counter(y_train)
    #add the confusion matrix and counter data in list
    #list is used in case we want to add more data, returing list will not result change of return type
    result_list.append(matrix)
    result_list.append(counter)
    return result_list

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [14]:
clf = BalancedRandomForestClassifier(max_depth=2, random_state=1)
model_name='BalancedRandomForestClassifier'
result_list = execute_classification_model(clf,
                                           model_name,
                                           X_train_scaled,
                                           y_train,
                                           X_test_scaled,
                                           y_test)
print(f"confusion matrix: \n",result_list[0])
print(f"Counter y_train: \n",result_list[1])

0.7223058204511484
                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.06      0.48      0.96      0.11      0.68      0.44        87
   low_risk       1.00      0.96      0.48      0.98      0.68      0.49     17118

avg / total       0.99      0.96      0.49      0.97      0.68      0.49     17205

confusion matrix: 
 [[   42    45]
 [  653 16465]]
Counter y_train: 
 Counter({'loan_status': 1})


In [15]:
# List the features sorted in descending order by feature importance
importances = clf.feature_importances_
sorted(zip(clf.feature_importances_, X.columns), reverse=True)

[(0.13358385419997146, 'total_rec_prncp'),
 (0.1282232009028145, 'last_pymnt_amnt'),
 (0.12092286830085001, 'total_pymnt_inv'),
 (0.09530847587913936, 'total_rec_int'),
 (0.07800042573623381, 'total_pymnt'),
 (0.03348265636625316, 'int_rate'),
 (0.029094369450241978, 'mths_since_recent_inq'),
 (0.021708325750352885, 'avg_cur_bal'),
 (0.019820958864326263, 'inq_last_12m'),
 (0.015847209766956076, 'annual_inc'),
 (0.015370884116032355, 'dti'),
 (0.014576255653144828, 'num_actv_rev_tl'),
 (0.01340486755307324, 'out_prncp'),
 (0.013044913748216473, 'revol_bal'),
 (0.012283052358052311, 'num_rev_tl_bal_gt_0'),
 (0.011668852674471654, 'total_rec_late_fee'),
 (0.011532172461510597, 'tot_coll_amt'),
 (0.01149876141581347, 'mths_since_rcnt_il'),
 (0.011444825298998231, 'num_actv_bc_tl'),
 (0.01131701143694686, 'tot_hi_cred_lim'),
 (0.011173465444731858, 'pct_tl_nvr_dlq'),
 (0.01114571467544235, 'issue_d_Jan-2019'),
 (0.011138883054152154, 'open_acc_6m'),
 (0.010065367298560222, 'total_bc_limit'

### Easy Ensemble Classifier

In [16]:
# Train the Classifier
eec = EasyEnsembleClassifier(random_state=1)
model_name='EasyEnsembleClassifier'
result_list = execute_classification_model(eec,
                                           model_name,
                                           X_train_scaled,
                                           y_train,
                                           X_test_scaled,
                                           y_test)
print(f"confusion matrix: \n",result_list[0])
print(f"Counter y_train: \n",result_list[1])




0.9259823295502616
                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.08      0.91      0.94      0.14      0.93      0.85        87
   low_risk       1.00      0.94      0.91      0.97      0.93      0.86     17118

avg / total       0.99      0.94      0.91      0.97      0.93      0.86     17205

confusion matrix: 
 [[   79     8]
 [  960 16158]]
Counter y_train: 
 Counter({'loan_status': 1})


In [17]:
print(f"model accuracy data:",balanced_accuracy_score_results)

model accuracy data: {'BalancedRandomForestClassifier': 0.7223058204511484, 'EasyEnsembleClassifier': 0.9259823295502616}


### Final Questions

1. Which model had the best balanced accuracy score?

    The Easy Ensemble Classifier has a better balance accuracy score of 0.9259823295502616 vs the Balanced Random Forest Classifier which had the balance accuracy score of 0.7223058204511484

2. Which model had the best recall score?

    The recall score of Balanced Random Forest Classifier is higher i.e 0.96 than Easy Ensemble Classifier which has recall score of 0.94. The higer recall score might be better in case we want to be risk averse and further analyse false positives

3. Which model had the best geometric mean score?

    The geometric score of Easy Ensemble Classifier is better i.e 0.93 as compare to Balanced Random Forest Classifier which as geometric score of 0.68

4. What are the top three features?

    The top 3 features based on feature importance from Balanced Random Forest Classifier are as follows
    1) Total received priciple
    2) Last Payment amount
    3) total payment invoice
    Result - 
      - (0.13358385419997146, 'total_rec_prncp'),
      - (0.1282232009028145, 'last_pymnt_amnt'),
      - (0.12092286830085001, 'total_pymnt_inv'),